In [ ]:
import os
os.chdir('C:\\Github')

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import euclidean_distances
import statistics
import pandas as pd 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import csv
from sklearn import tree
#import xgboost as xgb
from datetime import datetime
import pickle
from random import seed
from random import random

table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

def coor_to_class(s):
    return table[eval(s)[1]][eval(s)[0]]

def to_int(s):
    return int(s)

def class_to_coordinate(a):
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    coordinate = [x,y]
    return coordinate

def euclidean_distance(p1,p2):
    return np.sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]))

def uuid_to_b_num(s):
    if s[0:30] == 'e20a39f4-73f5-4bc4-a12f-17d1ad' :
        uuid_num = int(s[-1])
    else :
        uuid_num = 0
    return uuid_num

## 說明：
- 經由 app 收集而來的 BLE_data.csv 把每 10 秒收到的同個 beacon 的 RSSI 做平均產生 wireless fingerprint，每次一共會產生四個檔案，
- 在這一階段可以操作每台手機的 packet reception rate，以及使用的 beacon 數量
- 以下列檔案為例，"avg_10" 表示取 10 秒的 RSSI 做平均，"rate_7" 表示 packet reception rate=0.7
    1. raw_fingerprint_avg_10_rate_7.csv: 不做平均的 fingerprint，每個 step 可能會有多個 fingerprint，之後會用這個檔案來算真實的 packet reception rate，詳見 "Packet_loss_rate.ipynb"

    2. wireless_fingerprint_avg_10_**6_beacon**_rate_7.csv: 使用 6 個 beacon 的情況產生的 fingerprint

    3. wireless_fingerprint_avg_10_**4_beacon**_rate_7.csv: 使用 4 個 beacon 的情況產生的 fingerprint

    4. wireless_fingerprint_avg_10_**2_beacon**_rate_7.csv: 使用 2 個 beacon 的情況產生的 fingerprint


## 參數：
- loss_rate: packet reception rate, 要保留多少比例的封包，例如：loss_rate=0.7，表示會隨機忽略30%的封包

- sec: 一次要抓幾秒來算平均，因為我們收資料時每10秒走一步，所以設定一次抓10秒算平均

In [ ]:
# Input
loss_rate = 0.5
sec = 10
walks = ['stationary_1',\
        'freewalk_1','freewalk_2','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_7','freewalk_8','freewalk_9',\
        'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
        'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
        ]
phones = ['GalaxyA51','hTCU11','hTCU19e','sharp025']

In [ ]:
with open(f'./Data/0318_92589_train/min_rssi.pickle', 'rb') as f:
    min_rssi = pickle.load(f)
with open(f'./Data/0318_92589_train/max_rssi.pickle', 'rb') as f:
    max_rssi = pickle.load(f)

for walk in walks:

    test_dir = f'./Data/0318_92589_test'

    for phone in phones:

        wireless_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
        raw_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])

        start_end = pd.read_csv(f'{test_dir}/{walk}.csv', delimiter=',')
        start_end = start_end[start_end['phone'] == phone]

        BLE_df = pd.read_csv(f'{test_dir}/{walk}/{phone}/BLE_data.csv', delimiter=',')
        sensor_data = pd.read_csv(f'{test_dir}/{walk}/{phone}/sensor_data.csv', delimiter=',')

        # print(start_end)
        Min = min_rssi[phone]
        Max = max_rssi[phone]

        for idx, Row in start_end.iterrows(): # for every step

            time = sensor_data.copy()
            df = BLE_df.copy()

            avg_rssi = (df['RSSI'].mean()-Min)/(Max-Min)
            # print(avg_rssi)

            x = int(Row['position']/100)
            y = int(Row['position']%100)
            position = table[y][x]

            start_sec = Row['start']
            # print(start_sec)
            time['Time'] = pd.DatetimeIndex(time['Time'])
            init_time = time.iloc[0]['Time']
            start_time = init_time + pd.to_timedelta(start_sec, unit='s')
            end_time = start_time + pd.to_timedelta(sec, unit='s') # add 10 second

            df['Time'] = pd.DatetimeIndex(df['Time'])
            df.set_index(keys='Time', inplace=True)
            df = df.between_time(str(start_time)[11:], str(end_time)[11:])
            df = df.reset_index()
            df['UUID'] = df['UUID'].apply(uuid_to_b_num)

            # 整合為 beacon 形式
            rssi_time = []
            rssi = [np.zeros((8))]
            rssi[0][:] = np.nan 
            rssi[0][0] = position
            row = 0
            for i in range(len(df) - 1):
                curr = df.iloc[i]['Time']
                next = df.iloc[i+1]['Time']

                rssi_idx = int(df.iloc[i]['UUID'])

                if random() <= loss_rate:
                    rssi[row][rssi_idx] = (df.iloc[i]['RSSI']-Min)/(Max-Min)

                if curr != next:  
                    rssi_time.append(curr)  
                    row += 1
                    rssi.append(np.zeros((8)))
                    rssi[row][:] = np.nan
                    rssi[row][0] = position

            rssi_time.append(df.iloc[len(df) - 1]['Time'])
            rssi_idx = int(df.iloc[len(df) - 1]['UUID'])

            if random() <= loss_rate:
                rssi[row][rssi_idx] = (df.iloc[len(df) - 1]['RSSI']-Min)/(Max-Min)

            test_set = pd.DataFrame(np.array(rssi), columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
            test_set['Time'] = rssi_time
            
            # 取該段時間平均RSSI
            tmp = {'label':[position]} 
            for column in test_set.columns[1:]:
                tmp[column] = [test_set[column].mean()]

            testing_data = pd.DataFrame(tmp,columns=test_set.columns)

            wireless_fingerprint = wireless_fingerprint.append(testing_data,ignore_index=True)
            raw_fingerprint = raw_fingerprint.append(test_set,ignore_index=True)

            wireless_fingerprint = wireless_fingerprint.interpolate(limit_direction='both')

        #print(f'walk:{walk},phone:{phone},wireless_fingerprint:\n{wireless_fingerprint},raw_fingerprint:\n{raw_fingerprint}')

        wireless_fingerprint = wireless_fingerprint.drop(['Time'], axis='columns')
        wireless_6_beacon = wireless_fingerprint.drop(['Beacon_7'], axis='columns')
        wireless_5_beacon = wireless_fingerprint.drop(['Beacon_3','Beacon_4'], axis='columns')
        wireless_4_beacon = wireless_fingerprint.drop(['Beacon_3','Beacon_4','Beacon_7'], axis='columns')

        wireless_fingerprint.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_7_beacon_rate_{int(10*loss_rate)}.csv')
        
        wireless_6_beacon.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_6_beacon_rate_{int(10*loss_rate)}.csv')
        wireless_5_beacon.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_5_beacon_rate_{int(10*loss_rate)}.csv')
        wireless_4_beacon.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/wireless_fingerprint_avg_{sec}_4_beacon_rate_{int(10*loss_rate)}.csv')
        
        raw_fingerprint.set_index('label').to_csv(f'{test_dir}/{walk}/{phone}/raw_fingerprint_avg_{sec}_rate_{int(10*loss_rate)}.csv')